In [0]:
import pydicom
import tqdm

In [0]:
def getAccessionNbrAndExamCodeFromDicom(dcm):
    #dcm = pydicom.dcmread(dcm_filepath)
    try:
        nbr = dcm["AccessionNumber"].value
    except:
        nbr = None

    try:
        examcode = dcm["RequestedProcedureID"].value
    except:
        examcode = None

    return (nbr, examcode)
    

In [0]:
def accessionNbrToMillPersonId(accession_nbr):
    if accession_nbr is None:
        return None

    try:
        person_id = spark.sql(f"""
            SELECT MAX(MillPersonId) AS MillPersonId
            FROM 4_prod.pacs.all_pacs_ref_nbr
            WHERE refnbr = '{accession_nbr}'
        """).first()["MillPersonId"]
    except:
        person_id = None

    return person_id

In [0]:
dcm_filepath = "/Volumes/1_inland/sectra/vone/ben_test_1612-3_4b6af833fea9477b9aad1cd21032e876/000002/000001/000001.dcm"

# Read DICOMDIR
dcm = pydicom.dcmread(dcm_filepath)
accession_nbr, examcode = getAccessionNbrAndExamCodeFromDicom(dcm)
print(accession_nbr, examcode)

person_id = accessionNbrToMillPersonId(accession_nbr)
print(person_id)

In [0]:
def saveDicomWithNewPersonId(dcm, new_person_id, save_filepath):
    dcm["PatientID"].value = new_person_id
    dcm.save_as(save_filepath)

In [0]:
# Read DICOMDIR
dicomdir = pydicom.dcmread("/Volumes/1_inland/sectra/vone/ben_test_1612-3_4b6af833fea9477b9aad1cd21032e876/dicomdir")

for x in tqdm.tqdm(dicomdir.DirectoryRecordSequence):
    # Retrieve ReferencedFileID which contains dcm file path info if exists
    try:
        print(x["AccessionNumber"].value)
        
    except:
        continue

In [0]:
dicomdir